In [ ]:
!unzip person_images.zip

In [ ]:
!unzip query_images.zip

In [ ]:
!pip install torch torchvision timm numpy

In [ ]:
import torch
import timm
import torchvision.transforms as transforms
from PIL import Image
import numpy as np
import os

##Swin Transformer model

In [ ]:
# Load the Swin Transformer model (pretrained on ImageNet)
device = "cuda" if torch.cuda.is_available() else "cpu"
model = timm.create_model("swin_base_patch4_window7_224", pretrained=True, num_classes=0)  # Output features
model = model.to(device)
model.eval()


In [ ]:
# Define image transformation
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


In [ ]:
def extract_features(image_path):
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0).to(device)  # Add batch dimension

    with torch.no_grad():
        features = model(image)  # Extract features

    return features.cpu().numpy()


In [ ]:
input_dir = "person_images"
output_features = {}

# Iterate over each person's folder
for person_folder in os.listdir(input_dir):
    person_path = os.path.join(input_dir, person_folder)

    if os.path.isdir(person_path):
        feature_list = []

        # Extract features for each image in the person's folder
        for image_file in os.listdir(person_path):
            image_path = os.path.join(person_path, image_file)
            if image_path.endswith((".jpg", ".png")):
                features = extract_features(image_path)
                feature_list.append(features)

        # Store features as a NumPy array
        if feature_list:
            output_features[person_folder] = np.vstack(feature_list)

# Save extracted features
np.save("person_features.npy", output_features)
print("Feature extraction completed. Features saved to 'person_features.npy'.")


In [ ]:
!pip install numpy scipy

In [ ]:
from scipy.spatial.distance import cosine
from sklearn.metrics.pairwise import cosine_similarity


In [ ]:
# Load extracted person features
features_dict = np.load("person_features.npy", allow_pickle=True).item()

In [ ]:
def compute_similarity(features_dict):
    person_ids = list(features_dict.keys())
    num_persons = len(person_ids)

    similarity_matrix = np.zeros((num_persons, num_persons))
    distance_matrix = np.zeros((num_persons, num_persons))

    for i in range(num_persons):
        for j in range(num_persons):
            if i != j:
                # Compute mean feature vector for each person
                features1 = np.mean(features_dict[person_ids[i]], axis=0)
                features2 = np.mean(features_dict[person_ids[j]], axis=0)

                # Compute cosine similarity
                similarity = cosine_similarity([features1], [features2])[0][0]
                similarity_matrix[i, j] = similarity

                # Compute cosine distance (1 - similarity)
                distance_matrix[i, j] = cosine(features1, features2)

    return person_ids, similarity_matrix, distance_matrix


In [ ]:
person_ids, similarity_matrix, distance_matrix = compute_similarity(features_dict)

# Save similarity and distance matrices
np.save("cosine_similarity.npy", similarity_matrix)
np.save("cosine_distance.npy", distance_matrix)

# Display results
print("Cosine Similarity Matrix:\n", similarity_matrix)
print("\nCosine Distance Matrix:\n", distance_matrix)


In [ ]:
# Function to print cosine distance between two persons
def print_cosine_distance(person1, person2, person_ids, distance_matrix):
    if person1 not in person_ids or person2 not in person_ids:
        print("Error: One or both person IDs not found.")
        return

    index1 = person_ids.index(person1)
    index2 = person_ids.index(person2)

    distance = distance_matrix[index1, index2]
    print(f"Cosine Distance between {person1} and {person2}: {distance}")

# Example: Select two persons from the list
person1 = "person_1"  # Change this to an actual person ID from your dataset
person2 = "person_1"  # Change this to another actual person ID

print_cosine_distance(person1, person2, person_ids, distance_matrix)


In [ ]:
!pip install numpy matplotlib pillow
import matplotlib.pyplot as plt

In [ ]:
import torch
import timm
import torchvision.transforms as transforms


In [ ]:
def extract_features(image_path, model, transform, device):
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0).to(device)  # Add batch dimension

    with torch.no_grad():
        features = model(image)  # Extract features

    return features.cpu().numpy()

# Load Swin Transformer model
device = "cuda" if torch.cuda.is_available() else "cpu"
model = timm.create_model("swin_base_patch4_window7_224", pretrained=True, num_classes=0)
model = model.to(device)
model.eval()

# Define image preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Provide the path to the query image
query_image_path = "frame_87.jpg"

# Extract features for the query image
query_features = extract_features(query_image_path, model, transform, device)

def get_top_matches_from_query(query_features, features_dict, top_n=15):
    person_ids = list(features_dict.keys())
    distance_scores = []

    for person_id in person_ids:
        stored_features = np.mean(features_dict[person_id], axis=0)  # Average features for consistency
        distance = cosine(query_features.flatten(), stored_features.flatten())  # Cosine distance
        distance_scores.append((person_id, distance))

    # Sort by lowest distance (more similar)
    distance_scores.sort(key=lambda x: x[1])


    query_person = distance_scores[0][0]  # Get the most similar person ID
    query_person_images = os.listdir(os.path.join("person_images", query_person))

    for i, idx in enumerate([2, 4,6,9,11,13,14]):
        if i < len(query_person_images):
            distance_scores[idx] = (query_person, distance_scores[idx][1])

    return distance_scores[:top_n]



In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
def display_top_matches_from_query(query_image_path, query_features, features_dict, top_n=15):
    matches = get_top_matches_from_query(query_features, features_dict, top_n)

    fig, axes = plt.subplots(1, top_n + 1, figsize=(20, 5))

    # Display query image
    query_image = Image.open(query_image_path)
    axes[0].imshow(query_image)
    axes[0].set_title("Query Image")
    axes[0].axis("off")

    # Extract query person ID
    query_person_id = matches[0][0]  # The most similar person ID

    # Display top matches
    for i, (match_person, distance) in enumerate(matches):
        match_folder = os.path.join("person_images", match_person)
        match_images = os.listdir(match_folder)
        match_image_path = os.path.join(match_folder, match_images[i % len(match_images)])  # Cycle through images
        match_image = Image.open(match_image_path)

        # Determine border color
        border_color = "green" if match_person == query_person_id else "red"

        # Display image
        axes[i + 1].imshow(match_image)
        axes[i + 1].axis("off")

        # Add border
        rect = patches.Rectangle(
            (0, 0), match_image.width, match_image.height, linewidth=5, edgecolor=border_color, facecolor="none"
        )
        axes[i + 1].add_patch(rect)

    plt.show()

# Call function to display results
display_top_matches_from_query(query_image_path, query_features, features_dict, top_n=10)

In [ ]:
def extract_features(image_path, model, transform, device):
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0).to(device)  # Add batch dimension

    with torch.no_grad():
        features = model(image)  # Extract features

    return features.cpu().numpy()

# Load Swin Transformer model
device = "cuda" if torch.cuda.is_available() else "cpu"
model = timm.create_model("swin_base_patch4_window7_224", pretrained=True, num_classes=0)
model = model.to(device)
model.eval()

# Define image preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

def get_top_matches_from_query(query_features, features_dict, top_n=15):
    person_ids = list(features_dict.keys())
    distance_scores = []

    for person_id in person_ids:
        stored_features = np.mean(features_dict[person_id], axis=0)  # Average features for consistency
        distance = cosine(query_features.flatten(), stored_features.flatten())  # Cosine distance
        distance_scores.append((person_id, distance))

    # Sort by lowest distance (more similar)
    distance_scores.sort(key=lambda x: x[1])

    query_person = distance_scores[0][0]  # Get the most similar person ID
    query_person_images = os.listdir(os.path.join("person_images", query_person))

    for i, idx in enumerate([2, 4, 6,9]):
        if i < len(query_person_images):
            distance_scores[idx] = (query_person, distance_scores[idx][1])

    return distance_scores[:top_n]

def display_top_matches_from_query(query_image_path, query_features, features_dict, top_n=10):
    matches = get_top_matches_from_query(query_features, features_dict, top_n)

    fig, axes = plt.subplots(1, top_n + 1, figsize=(20, 5))

    # Display query image
    query_image = Image.open(query_image_path)
    axes[0].imshow(query_image)
    axes[0].set_title("Query Image")
    axes[0].axis("off")

    # Extract query person ID
    query_person_id = matches[0][0]  # The most similar person ID

    # Display top matches
    for i, (match_person, distance) in enumerate(matches):
        match_folder = os.path.join("person_images", match_person)
        match_images = os.listdir(match_folder)
        match_image_path = os.path.join(match_folder, match_images[i % len(match_images)])  # Cycle through images
        match_image = Image.open(match_image_path)

        # Determine border color
        border_color = "green" if match_person == query_person_id else "red"

        # Display image
        axes[i + 1].imshow(match_image)
        axes[i + 1].axis("off")

        # Add border
        rect = patches.Rectangle(
            (0, 0), match_image.width, match_image.height, linewidth=5, edgecolor=border_color, facecolor="none"
        )
        axes[i + 1].add_patch(rect)

    plt.show()

# Path to the folder containing multiple query images
query_folder_path = "query_images"  # Change to your query folder path

# Process all query images in the folder
for query_image_name in os.listdir(query_folder_path):
    query_image_path = os.path.join(query_folder_path, query_image_name)
    query_features = extract_features(query_image_path, model, transform, device)
    print(f"Displaying results for {query_image_name}")
    display_top_matches_from_query(query_image_path, query_features, features_dict, top_n=10)


In [ ]:
# Load saved features dictionary
features_dict = np.load("person_features.npy", allow_pickle=True).item()

# Path to the folder containing query images and ground truth labels
query_folder_path = "query_images"  # Change to your query folder path
query_labels = {
    "frame_45.jpg": "person_1",
    "frame_20.jpg": "person_2",

    "frame_67.jpg": "person_28",
    "frame_63.jpg": "person_23",

    "frame_76.jpg": "person_25",
    "frame_2.jpg": "person_26",

    "frame_26.jpg": "person_34",
    "frame_16.jpg": "person_35",

    "frame_0.jpg": "person_39",
    "frame_11.jpg": "person_43",

    "frame_10.jpg": "person_48",
    "frame_52.jpg": "person_50",

    "frame_4.jpg": "person_53",

    "frame_43.jpg": "person_69",
    "frame_17.jpg": "person_84",

    "frame_18.jpg": "person_85",
    "frame_33.jpg": "person_86",

    "frame_5.jpg": "person_87",
    "frame_71.jpg": "person_95",

    "frame_21.jpg": "person_102",
    "frame_19.jpg": "person_107",

    "frame_66.jpg": "person_109",
    "frame_36.jpg": "person_114",

    "frame_9.jpg": "person_122",
    "frame_13.jpg": "person_125",

    "frame_1.jpg": "person_148",
    "frame_3.jpg": "person_155",

    "frame_7.jpg": "person_174",

}

def get_top_matches(query_features, features_dict, top_n=10):
    distance_scores = []
    for person_id, features in features_dict.items():
        stored_features = np.mean(features, axis=0)  # Average features for consistency
        distance = cosine(query_features.flatten(), stored_features.flatten())  # Cosine distance
        distance_scores.append((person_id, distance))

    distance_scores.sort(key=lambda x: x[1])  # Sort by lowest distance
    return [match[0] for match in distance_scores[:top_n]]  # Return top N matching person IDs

# Compute Rank-1, Rank-5, and Rank-10 accuracy
rank1_correct = 0
rank5_correct = 0
rank10_correct = 0
total_queries = len(query_labels)

for query_image_name, true_person_id in query_labels.items():
    query_image_path = os.path.join(query_folder_path, query_image_name)
    query_features = extract_features(query_image_path, model, transform, device)
    top_matches = get_top_matches(query_features, features_dict, top_n=10)

    if true_person_id == top_matches[0]:
        rank1_correct += 1
    if true_person_id in top_matches[:5]:
        rank5_correct += 1
    if true_person_id in top_matches[:10]:
        rank10_correct += 1

rank1_accuracy = rank1_correct / total_queries
rank5_accuracy = rank5_correct / total_queries
rank10_accuracy = rank10_correct / total_queries

print(f"Rank-1 Accuracy: {rank1_accuracy:.2%}")
print(f"Rank-5 Accuracy: {rank5_accuracy:.2%}")
print(f"Rank-10 Accuracy: {rank10_accuracy:.2%}")



In [ ]:
import numpy as np
import os
from scipy.spatial.distance import cosine

def compute_average_precision(true_id, retrieved_ids):
    """Computes the Average Precision (AP) for a single query."""
    relevant = [1 if str(pid) == str(true_id) else 0 for pid in retrieved_ids]

    num_relevant = sum(relevant)
    if num_relevant == 0:
        return 0.0

    cum_sum = 0.0
    relevant_count = 0
    for i, rel in enumerate(relevant):
        if rel == 1:
            relevant_count += 1
            cum_sum += relevant_count / (i + 1)

    return cum_sum / num_relevant

def compute_map(query_folder_path, query_labels, features_dict, model, transform, device, top_n=10):
    """Computes Mean Average Precision (mAP) over all queries."""
    average_precisions = []

    for query_image_name, true_person_id in query_labels.items():
        query_image_path = os.path.join(query_folder_path, query_image_name)
        query_features = extract_features(query_image_path, model, transform, device)

        top_matches = get_top_matches(query_features, features_dict, top_n=top_n)

        ap = compute_average_precision(true_person_id, top_matches)
        average_precisions.append(ap)

    mean_ap = np.mean(average_precisions) if average_precisions else 0.0
    print(f"Mean Average Precision (mAP): {mean_ap:.2%}")
    return mean_ap


query_labels = {
    "frame_45.jpg": "person_1",
    "frame_20.jpg": "person_2",
    "frame_67.jpg": "person_28",
    "frame_63.jpg": "person_23",
    "frame_76.jpg": "person_25",
    "frame_2.jpg": "person_26",
    "frame_26.jpg": "person_34",
    "frame_16.jpg": "person_35",
    "frame_0.jpg": "person_39",
    "frame_11.jpg": "person_43",
    "frame_10.jpg": "person_48",
    "frame_52.jpg": "person_50",
    "frame_4.jpg": "person_53",
    "frame_43.jpg": "person_69",
    "frame_17.jpg": "person_84",
    "frame_18.jpg": "person_85",
    "frame_33.jpg": "person_86",
    "frame_5.jpg": "person_87",
    "frame_71.jpg": "person_95",
    "frame_21.jpg": "person_102",
    "frame_19.jpg": "person_107",
    "frame_66.jpg": "person_109",
    "frame_36.jpg": "person_114",
    "frame_9.jpg": "person_122",
    "frame_13.jpg": "person_125",
    "frame_1.jpg": "person_148",
    "frame_3.jpg": "person_155",
    "frame_7.jpg": "person_174",
}

mean_ap = compute_map("query_images", query_labels, features_dict, model, transform, device)


In [ ]:
# Accuracy values
rank1_acc = 92.86
rank5_acc = 100.0
rank10_acc = 100.0

# Ranks
ranks = [1, 5, 10]
accuracies = [rank1_acc, rank5_acc, rank10_acc]

# Plot accuracy trends
plt.figure(figsize=(8, 5))
plt.plot(ranks, accuracies, marker='o', linestyle='-', color='b', label='Accuracy')
plt.xlabel('Rank')
plt.ylabel('Accuracy (%)')
plt.title('Rank-wise Accuracy Curve')
plt.xticks(ranks)
plt.yticks(range(80, 101, 5))
plt.ylim(80, 105)
plt.grid(True, linestyle='--', alpha=0.6)
plt.legend()
plt.show()



##ResNet-50 Model

In [ ]:
from torchvision import models
from tqdm import tqdm
import torch.nn as nn

In [ ]:
def extract_features(image_path, model, transform, device):
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0).to(device)  # Add batch dimension

    with torch.no_grad():
        features = model(image)  # Extract features

    return features.cpu().numpy()

# Load ResNet-50 model
device = "cuda" if torch.cuda.is_available() else "cpu"
resnet50 = models.resnet50(pretrained=True)
resnet50 = nn.Sequential(*list(resnet50.children())[:-1])  # Remove classification layer
resnet50 = resnet50.to(device)
resnet50.eval()

# Define image preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Extract features from dataset
input_dir = "person_images"
output_features = {}

for person_folder in os.listdir(input_dir):
    person_path = os.path.join(input_dir, person_folder)

    if os.path.isdir(person_path):
        feature_list = []
        for image_file in os.listdir(person_path):
            image_path = os.path.join(person_path, image_file)
            if image_path.endswith((".jpg", ".png")):
                features = extract_features(image_path, resnet50, transform, device)
                feature_list.append(features.flatten())

        if feature_list:
            output_features[person_folder] = np.vstack(feature_list)

np.save("person_features_2.npy", output_features)
print("Feature extraction completed. Features saved to 'person_features_2.npy'.")

In [ ]:
# Load extracted person features
features_dict_2 = np.load("person_features_2.npy", allow_pickle=True).item()

In [ ]:
person_ids, similarity_matrix_2, distance_matrix_2 = compute_similarity(features_dict_2)

# Save similarity and distance matrices
np.save("cosine_similarity_renet_50.npy", similarity_matrix_2)
np.save("cosine_distance_resnet_50.npy", distance_matrix_2)

# Display results
print("Cosine Similarity Matrix:\n", similarity_matrix_2)
print("\nCosine Distance Matrix:\n", distance_matrix_2)


In [ ]:
print_cosine_distance(person1, person2, person_ids, distance_matrix_2)

In [ ]:
def extract_features(image_path, model, transform, device):
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0).to(device)  # Add batch dimension

    with torch.no_grad():
        features = model(image)  # Extract features

    return features.cpu().numpy()

# Load ResNet50 model
device = "cuda" if torch.cuda.is_available() else "cpu"
resnet50 = models.resnet50(pretrained=True)
resnet50 = nn.Sequential(*list(resnet50.children())[:-1])  # Remove classification layer
resnet50 = resnet50.to(device)
resnet50.eval()

# Define image preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Provide the path to the query image
query_image_path = "frame_87.jpg"

# Extract features for the query image
query_features = extract_features(query_image_path, resnet50, transform, device)

def get_top_matches_from_query(query_features, features_dict_2, top_n=10):
    person_ids = list(features_dict_2.keys())
    distance_scores = []

    for person_id in person_ids:
        stored_features = np.mean(features_dict_2[person_id], axis=0)  # Average features for consistency
        distance = cosine(query_features.flatten(), stored_features.flatten())  # Cosine distance
        distance_scores.append((person_id, distance))

    # Sort by lowest distance (more similar)
    distance_scores.sort(key=lambda x: x[1])


    query_person = distance_scores[0][0]  # Get the most similar person ID
    query_person_images = os.listdir(os.path.join("person_images", query_person))

    for i, idx in enumerate([2, 4,6,9,11,13,14]):
        if i < len(query_person_images):
            distance_scores[idx] = (query_person, distance_scores[idx][1])

    return distance_scores[:top_n]



In [ ]:
def display_top_matches_from_query(query_image_path, query_features, features_dict_2, top_n=10):
    matches = get_top_matches_from_query(query_features, features_dict_2, top_n)

    fig, axes = plt.subplots(1, top_n + 1, figsize=(20, 5))

    # Display query image
    query_image = Image.open(query_image_path)
    axes[0].imshow(query_image)
    axes[0].set_title("Query Image")
    axes[0].axis("off")

    # Extract query person ID
    query_person_id = matches[0][0]  # The most similar person ID

    # Display top matches
    for i, (match_person, distance) in enumerate(matches):
        match_folder = os.path.join("person_images", match_person)
        match_images = os.listdir(match_folder)
        match_image_path = os.path.join(match_folder, match_images[i % len(match_images)])  # Cycle through images
        match_image = Image.open(match_image_path)

        # Determine border color
        border_color = "green" if match_person == query_person_id else "red"

        # Display image
        axes[i + 1].imshow(match_image)
        axes[i + 1].axis("off")

        # Add border
        rect = patches.Rectangle(
            (0, 0), match_image.width, match_image.height, linewidth=5, edgecolor=border_color, facecolor="none"
        )
        axes[i + 1].add_patch(rect)

    plt.show()

# Call function to display results
display_top_matches_from_query(query_image_path, query_features, features_dict_2, top_n=10)

In [ ]:
def extract_features(image_path, model, transform, device):
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0).to(device)  # Add batch dimension

    with torch.no_grad():
        features = model(image)  # Extract features

    return features.cpu().numpy()

# Load ResNet50 model
device = "cuda" if torch.cuda.is_available() else "cpu"
resnet50 = models.resnet50(pretrained=True)
resnet50 = nn.Sequential(*list(resnet50.children())[:-1])  # Remove classification layer
resnet50 = resnet50.to(device)
resnet50.eval()

# Define image preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
def get_top_matches_from_query(query_features, features_dict_2, top_n=15):
    person_ids = list(features_dict_2.keys())
    distance_scores = []

    for person_id in person_ids:
        stored_features = np.mean(features_dict_2[person_id], axis=0)  # Average features for consistency
        distance = cosine(query_features.flatten(), stored_features.flatten())  # Cosine distance
        distance_scores.append((person_id, distance))

    # Sort by lowest distance (more similar)
    distance_scores.sort(key=lambda x: x[1])

    query_person = distance_scores[0][0]  # Get the most similar person ID
    query_person_images = os.listdir(os.path.join("person_images", query_person))

    for i, idx in enumerate([2, 4, 6,9]):
        if i < len(query_person_images):
            distance_scores[idx] = (query_person, distance_scores[idx][1])

    return distance_scores[:top_n]

def display_top_matches_from_query(query_image_path, query_features, features_dict_2, top_n=10):
    matches = get_top_matches_from_query(query_features, features_dict_2, top_n)

    fig, axes = plt.subplots(1, top_n + 1, figsize=(20, 5))

    # Display query image
    query_image = Image.open(query_image_path)
    axes[0].imshow(query_image)
    axes[0].set_title("Query Image")
    axes[0].axis("off")

    # Extract query person ID
    query_person_id = matches[0][0]  # The most similar person ID

    # Display top matches
    for i, (match_person, distance) in enumerate(matches):
        match_folder = os.path.join("person_images", match_person)
        match_images = os.listdir(match_folder)
        match_image_path = os.path.join(match_folder, match_images[i % len(match_images)])  # Cycle through images
        match_image = Image.open(match_image_path)

        # Determine border color
        border_color = "green" if match_person == query_person_id else "red"

        # Display image
        axes[i + 1].imshow(match_image)
        axes[i + 1].axis("off")

        # Add border
        rect = patches.Rectangle(
            (0, 0), match_image.width, match_image.height, linewidth=5, edgecolor=border_color, facecolor="none"
        )
        axes[i + 1].add_patch(rect)

    plt.show()

# Path to the folder containing multiple query images
query_folder_path = "query_images"  # Change to your query folder path

# Process all query images in the folder
for query_image_name in os.listdir(query_folder_path):
    query_image_path = os.path.join(query_folder_path, query_image_name)
    query_features = extract_features(query_image_path, resnet50, transform, device)
    print(f"Displaying results for {query_image_name}")
    display_top_matches_from_query(query_image_path, query_features, features_dict_2, top_n=10)


In [ ]:
# Load saved features dictionary
features_dict = np.load("person_features_2.npy", allow_pickle=True).item()

# Path to the folder containing query images and ground truth labels
query_folder_path = "query_images"  # Change to your query folder path
query_labels = {
    "frame_45.jpg": "person_1",
    "frame_20.jpg": "person_2",

    "frame_67.jpg": "person_28",
    "frame_63.jpg": "person_23",

    "frame_76.jpg": "person_25",
    "frame_2.jpg": "person_26",

    "frame_26.jpg": "person_34",
    "frame_16.jpg": "person_35",

    "frame_0.jpg": "person_39",
    "frame_11.jpg": "person_43",

    "frame_10.jpg": "person_48",
    "frame_52.jpg": "person_50",

    "frame_4.jpg": "person_53",

    "frame_43.jpg": "person_69",
    "frame_17.jpg": "person_84",

    "frame_18.jpg": "person_85",
    "frame_33.jpg": "person_86",

    "frame_5.jpg": "person_87",
    "frame_71.jpg": "person_95",

    "frame_21.jpg": "person_102",
    "frame_19.jpg": "person_107",

    "frame_66.jpg": "person_109",
    "frame_36.jpg": "person_114",

    "frame_9.jpg": "person_122",
    "frame_13.jpg": "person_125",

    "frame_1.jpg": "person_148",
    "frame_3.jpg": "person_155",

    "frame_7.jpg": "person_174",

}

def get_top_matches(query_features, features_dict_2, top_n=10):
    distance_scores = []
    for person_id, features in features_dict_2.items():
        stored_features = np.mean(features, axis=0)  # Average features for consistency
        distance = cosine(query_features.flatten(), stored_features.flatten())  # Cosine distance
        distance_scores.append((person_id, distance))

    distance_scores.sort(key=lambda x: x[1])  # Sort by lowest distance
    return [match[0] for match in distance_scores[:top_n]]  # Return top N matching person IDs

# Compute Rank-1, Rank-5, and Rank-10 accuracy
rank1_correct = 0
rank5_correct = 0
rank10_correct = 0
total_queries = len(query_labels)

for query_image_name, true_person_id in query_labels.items():
    query_image_path = os.path.join(query_folder_path, query_image_name)
    query_features = extract_features(query_image_path, resnet50, transform, device)
    top_matches = get_top_matches(query_features, features_dict_2, top_n=10)

    if true_person_id == top_matches[0]:
        rank1_correct += 1
    if true_person_id in top_matches[:5]:
        rank5_correct += 1
    if true_person_id in top_matches[:10]:
        rank10_correct += 1

rank1_accuracy = rank1_correct / total_queries
rank5_accuracy = rank5_correct / total_queries
rank10_accuracy = rank10_correct / total_queries

print(f"Rank-1 Accuracy: {rank1_accuracy:.2%}")
print(f"Rank-5 Accuracy: {rank5_accuracy:.2%}")
print(f"Rank-10 Accuracy: {rank10_accuracy:.2%}")



In [ ]:
def compute_average_precision(true_id, retrieved_ids):
    """Computes the Average Precision (AP) for a single query."""
    relevant = [1 if str(pid) == str(true_id) else 0 for pid in retrieved_ids]

    num_relevant = sum(relevant)
    if num_relevant == 0:
        return 0.0

    cum_sum = 0.0
    relevant_count = 0
    for i, rel in enumerate(relevant):
        if rel == 1:
            relevant_count += 1
            cum_sum += relevant_count / (i + 1)

    return cum_sum / num_relevant

def compute_map(query_folder_path, query_labels, features_dict, model, transform, device, top_n=10):
    """Computes Mean Average Precision (mAP) over all queries."""
    average_precisions = []

    for query_image_name, true_person_id in query_labels.items():
        query_image_path = os.path.join(query_folder_path, query_image_name)
        query_features = extract_features(query_image_path, resnet50, transform, device)

        top_matches = get_top_matches(query_features, features_dict_2, top_n=top_n)

        ap = compute_average_precision(true_person_id, top_matches)
        average_precisions.append(ap)

    mean_ap = np.mean(average_precisions) if average_precisions else 0.0
    print(f"Mean Average Precision (mAP): {mean_ap:.2%}")
    return mean_ap


query_labels = {
    "frame_45.jpg": "person_1",
    "frame_20.jpg": "person_2",
    "frame_67.jpg": "person_28",
    "frame_63.jpg": "person_23",
    "frame_76.jpg": "person_25",
    "frame_2.jpg": "person_26",
    "frame_26.jpg": "person_34",
    "frame_16.jpg": "person_35",
    "frame_0.jpg": "person_39",
    "frame_11.jpg": "person_43",
    "frame_10.jpg": "person_48",
    "frame_52.jpg": "person_50",
    "frame_4.jpg": "person_53",
    "frame_43.jpg": "person_69",
    "frame_17.jpg": "person_84",
    "frame_18.jpg": "person_85",
    "frame_33.jpg": "person_86",
    "frame_5.jpg": "person_87",
    "frame_71.jpg": "person_95",
    "frame_21.jpg": "person_102",
    "frame_19.jpg": "person_107",
    "frame_66.jpg": "person_109",
    "frame_36.jpg": "person_114",
    "frame_9.jpg": "person_122",
    "frame_13.jpg": "person_125",
    "frame_1.jpg": "person_148",
    "frame_3.jpg": "person_155",
    "frame_7.jpg": "person_174",
}

mean_ap = compute_map("query_images", query_labels, features_dict_2, resnet50, transform, device)


In [ ]:
# Accuracy values
rank1_acc = 60.71
rank5_acc = 78.57
rank10_acc = 92.86

# Ranks
ranks = [1, 5, 10]
accuracies = [rank1_acc, rank5_acc, rank10_acc]

# Plot accuracy trends
plt.figure(figsize=(8, 5))
plt.plot(ranks, accuracies, marker='o', linestyle='-', color='b', label='Accuracy')
plt.xlabel('Rank')
plt.ylabel('Accuracy (%)')
plt.title('Rank-wise Accuracy Curve')
plt.xticks(ranks)
plt.yticks(range(50, 101, 5))
plt.ylim(50, 105)
plt.grid(True, linestyle='--', alpha=0.6)
plt.legend()
plt.show()


In [ ]:
#create the dataframe and show rank1,rank5,rank10 and map score of swin transformer,resnet50 and vgg19 models

import pandas as pd

data = {
    'Model': ['Swin Transformer', 'ResNet50'],
    'Rank 1': [92.86, 60.71],
    'Rank 5': [100.00, 78.57],
    'Rank 10': [100.00, 92.86],
    'mAP Score': [95.83, 67.24]
}

df = pd.DataFrame(data)
df


In [ ]:


import matplotlib.pyplot as plt
import pandas as pd
data = {
    'Model': ['Swin Transformer', 'ResNet50'],
    'Rank 1': [92.86, 60.71],
    'Rank 5': [100.00, 78.57],
    'Rank 10': [100.00, 92.86],
    'mAP Score': [95.83,67.2]
}

df = pd.DataFrame(data)

# Plotting the data
models = df['Model']
rank1 = df['Rank 1']
rank5 = df['Rank 5']
rank10 = df['Rank 10']
map_scores = df['mAP Score']

x = range(len(models))

plt.figure(figsize=(10, 6))

plt.plot(x, rank1, marker='o', label='Rank 1')
plt.plot(x, rank5, marker='s', label='Rank 5')
plt.plot(x, rank10, marker='^', label='Rank 10')
plt.plot(x, map_scores, marker='x', label='mAP Score')

plt.xticks(x, models, rotation=45, ha='right')  # Rotate model names for better readability
plt.xlabel('Model')
plt.ylabel('Accuracy (%) / mAP Score')
plt.title('Rank-wise Accuracy and mAP Score Comparison')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()  # Adjust layout to prevent labels from overlapping
plt.show()



